In [3]:
import os
import utilities
from maths_objects import (
    Formulas, SiteHierarchies, DataSource, FrontMatter, FormulaTable, PageTabs, 
    FormulaTableTypeSimple, FormulaTableTypeCalculus, FormulaFile)
import pandas as pd
from collections import namedtuple

data_source = DataSource()
site_hierarchy = SiteHierarchies(data_source.site_hierarchies)
formulas_by_year = Formulas(data_source.formulas_by_year)
formulas_cumulative = Formulas(data_source.formulas_by_year_cumulative)

In [15]:
index_formulas = formulas_by_year.filter_by_function(lambda x: x['Category'] == 'Indices')

index_table = FormulaTable(index_formulas)
index_table.set_type(FormulaTableTypeSimple)

index_file = FormulaFile()
index_file.is_cumulative_by_year = True
index_file.add_formula_table(index_table)

index_file.path_in_hierarchy()

'NSW/Year 9/Formulas/By year cumulative/Indices'